In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import DPRContextEncoderTokenizer

In [ ]:
model_name = 'facebook/dpr-ctx_encoder-single-nq-base'
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained(model_name)

In [ ]:
text = [("How are you?", "I am fine."), ("What's up?", "Not much.")]

In [ ]:
tokens_info=context_tokenizer(text, return_tensors='pt', padding=True, \
                              truncation=True, max_length=256)

In [ ]:
print(tokens_info)

In [ ]:
from transformers import DPRContextEncoder

In [ ]:
encoder_model = 'facebook/dpr-ctx_encoder-single-nq-base'
context_encoder = DPRContextEncoder.from_pretrained(encoder_model)

In [ ]:
outputs=context_encoder(**tokens_info)

In [ ]:
outputs.pooler_output.shape

In [ ]:
def read_and_split_text(filename):
  with open(filename, 'r', encoding='utf-8') as file:
    text = file.read()
  paragraphs = text.split('\n')
  paragraphs = [para.strip() for para in paragraphs if len(para.strip()) > 0]
  return paragraphs

In [ ]:
paragraphs = read_and_split_text('companyPolicies.txt')

In [ ]:
def encode_contexts(text_list):
  embeddings = []
  for text in text_list:
    inputs = context_tokenizer(text, return_tensors='pt', padding=True, \
                               truncation=True, max_length=256)
    outputs = context_encoder(**inputs)
    embeddings.append(outputs.pooler_output)
    return torch.cat(embeddings).detach().numpy()

In [ ]:
context_embeddings = encode_contexts(paragraphs)

In [ ]:
!pip install faiss-cpu --no-cache

In [ ]:
import faiss

In [ ]:
embedding_dim = 7